In [1]:
import pathlib

import pandas as pd
from pycytominer import feature_select
from pycytominer.cyto_utils import output

In [2]:
# set path to normalized data
normalized_data_path = pathlib.Path(
    "../data/CP_scDINO_features/combined_CP_scDINO_norm.parquet"
).resolve(strict=True)

# set the outout file path
feature_selected_output_file_path = pathlib.Path(
    "../data/CP_scDINO_features/combined_CP_scDINO_norm_fs.parquet"
).resolve()

# read in the normalized data
normalized_data = pd.read_parquet(normalized_data_path)
print(normalized_data.shape)
normalized_data.head()

(188065, 3873)


,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,...,channel_DNA_cls_feature_91_scDINO,channel_DNA_cls_feature_92_scDINO,channel_DNA_cls_feature_93_scDINO,channel_DNA_cls_feature_94_scDINO,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO,channel_DNA_cls_feature_9_scDINO
0,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,7,...,-0.996898,-0.838599,-0.085856,0.080005,0.918828,1.373562,0.980376,-0.123520,2.401852,1.516202
1,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,9,...,0.612554,-0.134966,0.358887,0.486711,0.074517,-1.604183,2.407552,-0.835988,-0.264486,0.153676
2,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,10,...,-0.123170,0.288920,0.687862,-0.039333,-1.202663,0.373103,1.048282,-0.359857,0.659583,0.537619
3,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,11,...,-2.376413,-0.184088,0.934017,-0.800594,0.178355,-1.380925,-0.285477,0.211796,0.443178,1.129714
4,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,12,...,-0.825394,0.715618,-0.786748,-0.963768,-0.906530,0.786611,-1.693796,-1.694061,-0.228990,0.648714


In [3]:
# define operations to be performed on the data
# list of operations for feature select function to use on input profile
feature_select_ops = [
    "variance_threshold",
    "blocklist",
    "drop_na_columns",
    "correlation_threshold",
]

# Get columns that contain "Metadata"
metadata_features = normalized_data.columns[
    normalized_data.columns.str.contains("Metadata")
].tolist()

# get the feature columns
feature_columns = normalized_data.columns.difference(metadata_features).to_list()

In [4]:
feature_select_df = feature_select(
    normalized_data,
    operation=feature_select_ops,
    features=feature_columns,
)
# add "Metadata_" to the beginning of each column name in the list
manual_block_list = [
    x
    for x in normalized_data.columns
    if "bounding" in x.lower()
    or "Location_Center_Y" in x.lower()
    or "Location_Center_X" in x.lower()
]

feature_select_df.columns = [
    "Metadata_" + column if column in manual_block_list else column
    for column in feature_select_df.columns
]
print("Feature selection complete, saving to parquet file!")
# save features selected df as parquet file
output(
    df=feature_select_df,
    output_filename=feature_selected_output_file_path,
    output_type="parquet",
)
print("Features have been selected!")
# check to see if the shape of the df has changed indicating feature selection occurred
print(normalized_data.shape)
print(feature_select_df.shape)
print(f"{normalized_data.shape[1] - feature_select_df.shape[1]} features were removed.")
print(f"{feature_select_df.shape[1]} features remain.")
feature_select_df.head()

Feature selection complete, saving to parquet file!
Features have been selected!
(188065, 3873)
(188065, 2373)
1500 features were removed.
2373 features remain.


,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,...,channel_DNA_cls_feature_91_scDINO,channel_DNA_cls_feature_92_scDINO,channel_DNA_cls_feature_93_scDINO,channel_DNA_cls_feature_94_scDINO,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO,channel_DNA_cls_feature_9_scDINO
0,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,7,...,-0.996898,-0.838599,-0.085856,0.080005,0.918828,1.373562,0.980376,-0.123520,2.401852,1.516202
1,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,9,...,0.612554,-0.134966,0.358887,0.486711,0.074517,-1.604183,2.407552,-0.835988,-0.264486,0.153676
2,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,10,...,-0.123170,0.288920,0.687862,-0.039333,-1.202663,0.373103,1.048282,-0.359857,0.659583,0.537619
3,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,11,...,-2.376413,-0.184088,0.934017,-0.800594,0.178355,-1.380925,-0.285477,0.211796,0.443178,1.129714
4,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,12,...,-0.825394,0.715618,-0.786748,-0.963768,-0.906530,0.786611,-1.693796,-1.694061,-0.228990,0.648714
